In [1]:
from selenium import webdriver # Основной модуль веб-драйвера
from selenium.webdriver.firefox.options import Options # Модуль для браузера используем Firefox
from selenium.webdriver.support.ui import WebDriverWait # Модуль для ожидания условия
from selenium.webdriver.support import expected_conditions as EC # Модуль набора присетов для WebDriverWait
from selenium.webdriver.common.keys import Keys # Модуль симуляции клавиатуры
from selenium.webdriver.common.by import By # Определение местоположения элементов
from selenium.webdriver.support import expected_conditions as Error # Персональный try exept
from selenium.common.exceptions import TimeoutException 
import time # Модуль для работы с временем
import re # Модуль для работы с регулярными выражениями
import pandas as pd # Модуль для работы с базами данных

In [2]:
# Модуль работы браузера
options = Options()
options.add_argument('-start-maximized')  # Запуск браузера в полном окне
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:126.0) Gecko/20100101 Firefox/126.0') # User-Agent
driver = webdriver.Firefox(options=options)  # Драйвер который запускает экземпляр браузера
driver.get('https://www.wildberries.ru/') # открытие вебсайта
time.sleep(4) # Таймер задержки

In [3]:
# Поиск строки поиска и ввод запроса в строку
wait = WebDriverWait(driver, 10) # Ожидание прогрузки страницы
input = wait.until(EC.presence_of_element_located((By.ID, "searchInput"))) # Ищем строку поиска
# Вводим фразу поиска и нажимаем Enter
input.send_keys('процессоры амд 9') # Имитируем ввод запроса в строку поиска
input.send_keys(Keys.ENTER) # Имитируем нажание кнопки ввода

In [4]:
# Модуль создания списка, прокручивания сайта и подсчёта карточек товара, парсинга и перехода на следующию страницу
product_list = [] # Список процессоров
# Прокручиваем сайт до конца
while True:
    count = None # Для подсчёта количества карточек товара
    while True:
        time.sleep(4) # Таймер задержки
        cards = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@id]'))) # Ищем карточку товара
        
        if len(cards) == count: # Выходим из цикла, если при прокрутке страницы, количество товаров не меняется
            break

        count = len(cards) # Посчитываем количество карточек товара на странице
        
        driver.execute_script('window.scrollBy(0, 1800)') # Прокручиваем страницу выполняя JAVA Script
        time.sleep(2) # Таймер задержки
    # Проходимся по карточкам, извлекаем ссылку на товар и добавляем в product_list    
    for card in cards:
        url = card.find_element(By.XPATH, './div/a').get_attribute('href')
        product_list.append(url)

    try: # Доработать проверку на ошибки!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        next = driver.find_element(By.XPATH,  "//a[@class='pagination-next pagination__next j-next-page']") # Ищем кнопку перехода на следующию страницу
        next.click()
    except Exception: 
        break


In [5]:
print(f'Всего получено: {len(product_list)} ссылок на процессоры AMD R9')

Всего получено: 72 ссылок на процессоры AMD R9


In [46]:
# Модуль парсинга данных с страницы товара
driver2 = webdriver.Firefox(options=options)  # Ещё один экземпляр браузера FireFox
wait2 = WebDriverWait(driver2, 10) # Таймер ожидания действий driver2
data_list = [] # Лист данных о процессорах

# Парсинг данных
for url_item in product_list:
    data_parsing = {} # Словарь для парсинга

    driver2.get(url_item) # Переход по страницам
    data_parsing['name'] = wait2.until(EC.presence_of_element_located((By.XPATH, "//h1"))).text # Парсим название процессора







    price = wait2.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'price-block__final-price')))
    for price_element in price:
        print(price_element.text)
    try:
        data_list['price'] = float(re.sub(r'[^\d.]+', '', price.text))
    except Exception:
        data_list['price'] = None


    # prices = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="price-block__content"]/div/p/span/ins[@class="price-block__final-price"]')))

    


    

    # try:
    #     price = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@class, "price-block__wallet-price")]')))
    #     # price = wait2.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'price-block__wallet-price')))
    #     data_parsing['price_WB_wallet_sales'] = float(re.sub(r'[^\d.]+', '', price[1].text))
    # except TimeoutException:
    #     data_parsing['price_WB_wallet_sales'] = '-'
    # except Exception as e:
    #     data_parsing['price_WB_wallet_sales'] = '-'









    # Блок парсинга цены, с скидкой, без, старой ценой, новой ценой и тд
    # price = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@class, "price-block__wallet-price")]')))

    # price = wait2.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'price-block__wallet-price'))) # парсим цену процессора, WB кошелёк распродажа
    # try: 
    #     data_parsing['price_WB_wallet_sales'] = float(re.sub(r'[^\d.]+', '', price[1].text))
    # except Exception:
    #     data_parsing['price_WB_wallet_sales'] = '-'
    try:
        price = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@class, "price-block__final-price wallet")]')))
        # price = wait2.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'price-block__final-price wallet'))) # парсим цену процессора, WB кошелёк старая цена
        data_parsing['price_WB_wallet_old'] = float(re.sub(r'[^\d.]+', '', price[1].text))
    except TimeoutException:
        data_parsing['price_WB_wallet_old'] = '-'
    except Exception:
        data_parsing['price_WB_wallet_old'] = '-'
    # price = wait2.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'price-block__final-price wallet'))) # парсим цену процессора, WB кошелёк старая цена
    # try: 
    #     data_parsing['price_WB_wallet_old'] = float(re.sub(r'[^\d.]+', '', price[1].text))
    # except Exception:
    #     data_parsing['price_WB_wallet_old'] = '-'
    try:
        price = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@class, "price-block__final-price") and not(contains(@class, "wallet"))]')))
        # price = wait2.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'price-block__final-price wallet'))) # парсим цену процессора, цена распродажа
        data_parsing['price_sales'] = float(re.sub(r'[^\d.]+', '', price[1].text))
    except TimeoutException:
        data_parsing['price_sales'] = '-'
    except Exception:
        data_parsing['price_sales'] = '-'
    # price = wait2.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'price-block__final-price'))) # парсим цену процессора, цена распродажа
    # try: 
    #     data_parsing['price_sales'] = float(re.sub(r'[^\d.]+', '', price[1].text))
    # except Exception:
    #     data_parsing['price_sales'] = '-'  
    try:
        price = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@class, "price-block__old-price")]')))
        # price = wait2.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'price-block__old-price'))) # парсим цену процессора, старая цена до распродажи
        data_parsing['price_old'] = float(re.sub(r'[^\d.]+', '', price[1].text))
    except TimeoutException:
        data_parsing['price_old'] = '-'
    except Exception:
        data_parsing['price_old'] = '-'
    # price = wait2.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'price-block__old-price'))) # парсим цену процессора, старая цена до распродажи
    # try: 
    #     data_parsing['price_old'] = float(re.sub(r'[^\d.]+', '', price[1].text))
    # except Exception:
    #     data_parsing['price_old'] = '-'

    data_parsing['brend'] = wait2.until(EC.presence_of_element_located((By.CLASS_NAME, "product-page__header-brand"))).text # Парсим бренд процессора
    data_parsing['url'] = url_item # Ссылка на процессор

    button = WebDriverWait(driver2, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'product-page__btn-detail')) # Находим клабельный элемент "Все характеристики и описание" чтобы получить открыть таблицу с данными
    )
    button.click() # Имитируем клик на кнопку "Все характеристики и описание"

    # Обрабатываем табличные данные
    table_row_name = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="popup popup-product-details shown"]//th')))
    table_row_param = wait2.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="popup popup-product-details shown"]//td')))
    # Заносим данные в зависимости от названия
    for i in range(len(table_row_name)):
        if table_row_name[i].text == 'Процессор':
            try:
                data_parsing['processor'] = table_row_param[i].text
            except Exception:
                data_parsing['processor'] = "-"
        elif table_row_name[i].text == 'Линейка процессоров':
            try:
                data_parsing['family_processor'] = table_row_param[i].text
            except Exception:
                data_parsing['family_processor'] = "-"
        elif table_row_name[i].text == 'Сокет':
            try:
                data_parsing['soked'] = table_row_param[i].text
            except Exception:
                data_parsing['soked'] = "-"
        elif table_row_name[i].text == 'Тактовая частота процессора':
            try:
                val = table_row_param[i].text.strip()
                val, *_ = val.split()
                data_parsing['cpu_clock_speed'] = float(re.sub(r'[^\d.]+', '', val))
                # data_parsing['cpu_clock_speed'] = table_row_param[i].text
            except Exception:
                data_parsing['cpu_clock_speed'] = "-"
        elif table_row_name[i].text == 'Максимальная частота в турбо режиме':
            try:
                val = table_row_param[i].text.strip()
                val, *_ = val.split()
                data_parsing['bost_cpu_clock_speed'] = float(re.sub(r'[^\d.]+', '', val))
                # data_parsing['bost_cpu_clock_speed'] = table_row_param[i].text
            except Exception:
                data_parsing['bost_cpu_clock_speed'] = "-"
        elif table_row_name[i].text == 'Количество ядер процессора':
            try:
                val = table_row_param[i].text.strip()
                val, *_ = val.split()
                data_parsing['processor_cores'] = float(re.sub(r'[^\d.]+', '', val))
                # data_parsing['processor_cores'] = table_row_param[i].text
            except Exception:
                data_parsing['processor_cores'] = "-"
        elif table_row_name[i].text == 'Максимальное число потоков':
            try:
                val = table_row_param[i].text.strip()
                val, *_ = val.split()
                data_parsing['Max_count_threads'] = float(re.sub(r'[^\d.]+', '', val))
                # data_parsing['Max_count_threads'] = table_row_param[i].text
            except Exception:
                data_parsing['Max_count_threads'] = "-"
        elif table_row_name[i].text == 'Техпроцесс':
            try:
                val = table_row_param[i].text.strip()
                val, *_ = val.split()
                data_parsing['technical_process'] = float(re.sub(r'[^\d.]+', '', val))
                # data_parsing['technical_process'] = table_row_param[i].text
            except Exception:
                data_parsing['technical_process'] = "-"
        elif table_row_name[i].text == 'Встроенная графическая система':
            try:
                data_parsing['processor_graphic'] = table_row_param[i].text
            except Exception:
                data_parsing['processor_graphic'] = "-"
        elif table_row_name[i].text == 'Объем кэша L3':
            try: 
                val = table_row_param[i].text.strip()
                val, *_ = val.split()
                data_parsing['cashe_L3'] = float(re.sub(r'[^\d.]+', '', val))
            except Exception:
                data_parsing['cashe_L3'] = None
        elif table_row_name[i].text == 'Страна производства':
            try: 
                data_parsing['counry'] = table_row_param[i].text
            except Exception:
                data_parsing['counry'] = None

    data_list.append(data_parsing) # Добавляем спарсеные значения в базу

57 346 ₽



TypeError: list indices must be integers or slices, not str

In [15]:
print(f'Обработано {len(data_list)} страниц')

Обработано 1 страниц


In [44]:
df = pd.DataFrame(data_list)
df.head()

,name,price_WB_wallet_old,price_sales,price_old,brend,url,processor,family_processor,soked,cpu_clock_speed,processor_cores,Max_count_threads,bost_cpu_clock_speed,technical_process,cashe_L3,processor_graphic,counry
0,Процессор RYZEN 9 7950X3D OEM (100-000000908),-,-,-,AMD,https://www.wildberries.ru/catalog/182999045/d...,7950X3D,Ryzen 9,AM5,4200.0,16.0,32.0,5700.0,5.0,128.0,AMD Radeon Graphics,Китай
